In [1]:
import yfinance as yf
import requests
import pandas as pd
import datetime as dt
import time
from datetime import timedelta
from prod.model import *
from functools import reduce
from yahoo_fin import stock_info as si
import numpy as np
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets


In [46]:
# inversion inicial en dolares
#initial_amount=5249000/777.18
initial_amount=5000000/777.18

# ponderacion del portafolio
ponderacion={'ETHO':4.6/100,
'ESGV':4.12/100,
'VT':12.16/100,
'XLK':15.16/100,
'VTI':19.61/100,
'IVV':11.42/100,
'QQQ':15.30/100,
'ESGU':4.79/100,
'SMH':6.72/100,
'ARKW':2.30/100,
'ARKK':2.13/100,
'ESGU':4.79/100,
'OTROS':2.23/100}

# portafolio en el que se invierte
tickers=["ESGV","ETHO","VT","XLK","VTI","IVV","QQQ","SMH","ARKW","ARKK","ESGU"]

In [47]:
dataframes=[]
for i in tickers:
       
    # Generamos el diccionario pada cada ticker
    string="{i}=ticker('{i}')".format(i=i)
    exec(string)
    
    string2="df_{i}={i}.getLatest()".format(i=i)
    exec(string2)
    
    string3="df_{i}['ticker']='{i}'".format(i=i)
    exec(string3)
    
    string4="dataframes.append(df_{i})".format(i=i)
    exec(string4)

# Concatenamos todos los dataframes en solo uno
day_display=reduce(lambda x, y: pd.concat([x, y]), dataframes)
day_display

Creating ticker ...
ESGV was created, date:08/12/2020
Creating ticker ...
ETHO was created, date:08/12/2020
Creating ticker ...
VT was created, date:08/12/2020
Creating ticker ...
XLK was created, date:08/12/2020
Creating ticker ...
VTI was created, date:08/12/2020
Creating ticker ...
IVV was created, date:08/12/2020
Creating ticker ...
QQQ was created, date:08/12/2020
Creating ticker ...
SMH was created, date:08/12/2020
Creating ticker ...
ARKW was created, date:08/12/2020
Creating ticker ...
ARKK was created, date:08/12/2020
Creating ticker ...
ESGU was created, date:08/12/2020


,date,open,high,low,last_price,change,price_change,ticker
0,2020-08-12,61.459999,61.834999,61.369999,61.680000,0.357958,0.220001,ESGV
0,2020-08-12,44.880001,44.990002,44.785000,44.810001,-0.155971,-0.070000,ETHO
0,2020-08-12,81.559998,82.029999,81.530998,81.870003,0.380095,0.310005,VT
0,2020-08-12,112.279999,114.206001,112.139999,113.910004,1.451732,1.630005,XLK
0,2020-08-12,170.369995,171.518997,170.339996,171.160004,0.463702,0.790009,VTI
0,2020-08-12,336.850006,339.640289,336.850006,338.679993,0.543265,1.829987,IVV
0,2020-08-12,267.369995,272.839996,267.369995,271.859985,1.679317,4.489990,QQQ
0,2020-08-12,168.720001,173.000000,168.029999,172.500000,2.240398,3.779999,SMH
0,2020-08-12,97.800003,98.930000,97.510002,98.400002,0.613495,0.599998,ARKW
0,2020-08-12,82.930000,84.289001,82.460098,83.949997,1.229949,1.019997,ARKK


In [48]:
# agregamos una columna de ponderacion de cada instrumento
day_display["pond"]=day_display["ticker"].apply(lambda x: ponderacion.get(x))

# Extraemos los montos que no tuvieron pnderacion
not_considered_per=1-sum(day_display["pond"])

# Guardamos lo que no se invirtio.. no se considera en la estimacion
rest=initial_amount*not_considered_per
invested=initial_amount-rest

# Agregamos columnas de cantidad inicial invertida y no invertida independiente del fondo
day_display["initial_invested_usd"]=initial_amount
day_display["real_invested"]=invested
day_display["not_invested"]=rest

# Calculamos lo que se invirtio en cada instrumento
day_display["invested"]=day_display["pond"].apply(lambda x: x*initial_amount)

# Calculamos las ganancias o perdidas
day_display["profit"]=(day_display["change"]/100)*(day_display["invested"])

# Calculamos el nuevo valor de la cantidad invertida en el instrumento 
day_display["estimated"]=day_display["invested"]+day_display["profit"]

# Agregamos una columna con el resultado final de lo invertido
day_display['total_result']=sum(day_display["estimated"])-sum(day_display["invested"])

In [49]:
day_display

,date,open,high,low,last_price,change,price_change,ticker,pond,initial_invested_usd,real_invested,not_invested,invested,profit,estimated,total_result
0,2020-08-12,61.459999,61.834999,61.369999,61.680000,0.357958,0.220001,ESGV,0.0412,6433.516045,6324.789624,108.726421,265.060861,0.948808,266.009669,58.133286
0,2020-08-12,44.880001,44.990002,44.785000,44.810001,-0.155971,-0.070000,ETHO,0.0460,6433.516045,6324.789624,108.726421,295.941738,-0.461583,295.480155,58.133286
0,2020-08-12,81.559998,82.029999,81.530998,81.870003,0.380095,0.310005,VT,0.1216,6433.516045,6324.789624,108.726421,782.315551,2.973540,785.289091,58.133286
0,2020-08-12,112.279999,114.206001,112.139999,113.910004,1.451732,1.630005,XLK,0.1516,6433.516045,6324.789624,108.726421,975.321032,14.159049,989.480082,58.133286
0,2020-08-12,170.369995,171.518997,170.339996,171.160004,0.463702,0.790009,VTI,0.1961,6433.516045,6324.789624,108.726421,1261.612496,5.850118,1267.462615,58.133286
0,2020-08-12,336.850006,339.640289,336.850006,338.679993,0.543265,1.829987,IVV,0.1142,6433.516045,6324.789624,108.726421,734.707532,3.991405,738.698938,58.133286
0,2020-08-12,267.369995,272.839996,267.369995,271.859985,1.679317,4.489990,QQQ,0.1530,6433.516045,6324.789624,108.726421,984.327955,16.529988,1000.857943,58.133286
0,2020-08-12,168.720001,173.000000,168.029999,172.500000,2.240398,3.779999,SMH,0.0672,6433.516045,6324.789624,108.726421,432.332278,9.685962,442.018240,58.133286
0,2020-08-12,97.800003,98.930000,97.510002,98.400002,0.613495,0.599998,ARKW,0.0230,6433.516045,6324.789624,108.726421,147.970869,0.907794,148.878663,58.133286
0,2020-08-12,82.930000,84.289001,82.460098,83.949997,1.229949,1.019997,ARKK,0.0213,6433.516045,6324.789624,108.726421,137.033892,1.685447,138.719339,58.133286


In [54]:
def save_gsheets(page,dataframe):
    gc = pygsheets.authorize(service_file='key.json')

    #open the google spreadsheet 
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1z7zzHgM6t4ySUkPZe5BdFsBvTD6RxsfSdKEnqWU1a2Q/edit#gid=67598709')

    #select the first sheet 
    wks = sh[page]

    #update the first sheet with df, starting at cell B2. 
    wks.set_dataframe(dataframe, 'A1')

In [56]:
save_gsheets(0,day_display)